In [ ]:
import requests
from bs4 import BeautifulSoup

def obtener_links_pokemon():
    url = "https://www.wikidex.net/wiki/Lista_de_Pokémon"
    response = requests.get(url)
    soup = BeautifulSoup(response.text)


    tablas = soup.find_all('table', class_='tabpokemon')
    links_pokemon = []
    for tabla in tablas:
        filas = tabla.find_all('tr')
        for fila in filas[1:]:
            celdas = fila.find_all('td')
            if not celdas:
                continue
            if len(celdas) == 3:
                celda_nombre = celdas[0]
            elif len(celdas) > 3:
                celda_nombre = celdas[1]
            else:
                # Si la fila no cumple estas condiciones, la saltamos
                continue
            enlace_tag = celda_nombre.find('a')

            # Comprobar que la etiqueta <a> y el atributo href existan
            if enlace_tag and enlace_tag.has_attr('href'):
                enlace = enlace_tag['href']
                links_pokemon.append("https://www.wikidex.net" + enlace)

    return links_pokemon


links_pokemon = obtener_links_pokemon()[:251]
for i, link in enumerate(links_pokemon):
  print(f'{i+1}: {link}')

print(len(links_pokemon))

In [ ]:
import requests
from bs4 import BeautifulSoup

TIPOS_ES = {
    "acero","agua","bicho","dragón","eléctrico","fantasma","fuego","hada","hielo",
    "lucha","normal","planta","psíquico","roca","siniestro","tierra","veneno","volador"
}

def extraer_estadisticas_y_tipos(url):
    r = requests.get(url)
    if r.status_code != 200:
        print("Error", r.status_code, url)
        return None

    soup = BeautifulSoup(r.text, "html.parser")

    # 1) Estadísticas (PS, Ataque, Defensa, Ataque Especial, Defensa Especial, Velocidad) 
    # CAMBIO EN LOS NOMBRES DE ATAQUE ESPECIAL Y DEFENSA ESPECIAL, ANTES NO LOS COGÍA BIEN
    stats = {
        "PS": None, "Ataque": None, "Defensa": None,
        "At. esp.": None, "Def. esp.": None, "Velocidad": None
    }

    # Buscamos cualquier tabla que contenga filas TH/TD con esos nombres
    for tabla in soup.find_all("table"):
        filas = tabla.find_all("tr")
        for tr in filas:
            celdas = tr.find_all(["th","td"])
            if len(celdas) < 2:
                continue
            clave = celdas[0].get_text(" ", strip=True)
            valor = celdas[1].get_text(" ", strip=True)

            # Comprobamos si el contenido de la clave es una de las estadísticas que buscamos
            if clave in stats and (stats[clave] is None):
                # nos quedamos con el primer valor que encontremos
                # quitamos posibles símbolos y convertimos a int si se puede
                valnum = ''.join(ch for ch in valor if ch.isdigit())
                stats[clave] = int(valnum) if valnum.isdigit() else None

    # 2) Tipos (heurística por texto del enlace)  CAMBIADO, ANTES NO COGÍA BIEN TODOS LOS TIPOS
    tipos = []
    for span in soup.find_all("span", typeof="mw:File"):
        a = span.find("a")
        if a and a.has_attr("title"):
            tipo = a["title"].replace("Tipo ", "").lower()
            if tipo in TIPOS_ES and tipo not in tipos:
                tipos.append(tipo)

    # Normalizar: máximo 2 tipos
    tipos = tipos[:2]

    # 3) Nombre (del título o de la URL)
    nombre = soup.find("h1").get_text(" ", strip=True) if soup.find("h1") else url.rstrip("/").split("/")[-1]

    return {
        "nombre": nombre,
        "tipos": tipos,
        "ps": stats["PS"],
        "ataque": stats["Ataque"],
        "defensa": stats["Defensa"],
        "ataque_esp": stats["At. esp."],
        "defensa_esp": stats["Def. esp."],
        "velocidad": stats["Velocidad"],
        "url": url
    }

In [ ]:
# Equipo defensivo
import requests
from bs4 import BeautifulSoup
def obtener_pokemons_fuego():
    url = "https://www.wikidex.net/wiki/Lista_de_Pokémon"
    response = requests.get(url)
    soup = BeautifulSoup(response.text)

    pokemons_fuego = []

    tablas = soup.find_all('table', class_='tabpokemon')
    
    for tabla in tablas:
        filas = tabla.find_all('tr')
        for fila in filas[1:]:
            celdas = fila.find_all('td')
            if not celdas:
                continue

            # Pokemon de Alola. Contenido de celdas: 0:nombre, 1:tipos, 2:japonés
            if len(celdas) == 3:
                celda_nombre = celdas[0]

                celda_tipos = celdas[1]

            # Pokemon normal. Contenido de celdas: 0:número, 1:nombre, 2:tipos, 3:japonés
            elif len(celdas) > 3:
                
                celda_nombre = celdas[1]
                celda_tipos = celdas[2]     

            else:
                continue

            # Nombre y link del pokemon
            enlace_tag = celda_nombre.find('a')

            # Comprobar que la etiqueta <a> y el atributo href existan
            if enlace_tag and enlace_tag.has_attr('href'):
                enlace = enlace_tag['href']
                link_pokemon = "https://www.wikidex.net" + enlace

            if enlace_tag and enlace_tag.has_attr('title'):
                nombre = enlace_tag['title']

            # Tipos del pokemon
            tipos = []
            for span in celda_tipos.find_all("span", typeof="mw:File"):
                a = span.find("a")
                if a and a.has_attr("title"):
                    # Guardamos "Tipo Volador" como "volador"
                    tipo = a["title"].replace("Tipo ", "").lower()  
                    tipos.append(tipo)
            
            if "fuego" in tipos:
                pokemons_fuego.append({
                    "nombre": nombre,
                    "link": link_pokemon,
                    "tipos": tipos
                })

    return pokemons_fuego


In [ ]:
# Visualizar lista de pokemons de tipo fuego
pokemons_fuego = obtener_pokemons_fuego()

print(f"Se han encontrado {len(pokemons_fuego)} pokémons de tipo fuego.")
i = 1
for pokemon in pokemons_fuego:
    print(f"{i} {pokemon['nombre']} ({', '.join(pokemon['tipos'])}): {pokemon['link']}")
    i += 1

In [ ]:
for pokemon in pokemons_fuego:
    print(extraer_estadisticas_y_tipos(pokemon['link']))